In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torch.nn import functional as F


import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")
import numpy as np
import torchutils as tu
import os
import json
import torchvision

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [14]:
def get_classes(data_dir):
    classes = sorted(os.listdir(data_dir))
    return classes

In [19]:
test_classes = get_classes("/home/artemiy/nn_project/data/seg_test/seg_test")
train_classes = get_classes("/home/artemiy/nn_project/data/seg_train/seg_train")

In [20]:
print(test_classes)
print(train_classes)


['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [35]:
def create_class_index(data_dir):
    classes = get_classes(data_dir)
    class_index = {i: class_name for i, class_name in enumerate(classes)}
    return class_index

In [36]:
train_dir = "/home/artemiy/nn_project/data/seg_train/seg_train"
test_dir = "/home/artemiy/nn_project/data/seg_test/seg_test"

In [37]:
train_classes = get_classes(train_dir)
test_classes = get_classes(test_dir)

In [38]:
# Создание и сохранение словаря классов
class_index = create_class_index(train_dir)
with open("class_index.json", "w") as f:
    json.dump(class_index, f)

print("Class index:", class_index)

Class index: {0: 'buildings', 1: 'forest', 2: 'glacier', 3: 'mountain', 4: 'sea', 5: 'street'}


In [47]:
labels = json.load(open("class_index.json"))
decode = lambda x: labels[str(x)] if str(x) in labels else 'Мы не знаем что это такое если бы мы знали мы не знаем что это такое'


In [40]:
# Проверка функции декодирования
for i in range(len(labels)):
    print(f"Index {i} corresponds to class {decode(i)}")

Index 0 corresponds to class buildings
Index 1 corresponds to class forest
Index 2 corresponds to class glacier
Index 3 corresponds to class mountain
Index 4 corresponds to class sea
Index 5 corresponds to class street


In [46]:
decode(15)

'Мы не знаем что это такое если бы мы знали мы не знаем что это такое ымпп укцрукер цкерц'

In [51]:
from torchvision.models import Inception_V3_Weights, inception_v3
model = inception_v3(weights=Inception_V3_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/artemiy/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100.0%


In [54]:
model.to(DEVICE)
tu.get_model_summary(model, torch.randn(3, 3, 299, 299, device=DEVICE))

Layer                                              Kernel              Output          Params             FLOPs
0_Conv2d_1a_3x3.Conv2d_conv                       [3, 32, 3, 3]   [3, 32, 149, 149]         864      57,544,992
1_Conv2d_1a_3x3.BatchNorm2d_bn                             [32]   [3, 32, 149, 149]          64       8,525,184
2_Conv2d_2a_3x3.Conv2d_conv                      [32, 32, 3, 3]   [3, 32, 147, 147]       9,216     597,445,632
3_Conv2d_2a_3x3.BatchNorm2d_bn                             [32]   [3, 32, 147, 147]          64       8,297,856
4_Conv2d_2b_3x3.Conv2d_conv                      [32, 64, 3, 3]   [3, 64, 147, 147]      18,432   1,194,891,264
5_Conv2d_2b_3x3.BatchNorm2d_bn                             [64]   [3, 64, 147, 147]         128      16,595,712
6_maxpool1                                                    -     [3, 64, 73, 73]           0               0
7_Conv2d_3b_1x1.Conv2d_conv                      [64, 80, 1, 1]     [3, 80, 73, 73]       5,120      81,

In [55]:
transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(), 
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [56]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

In [57]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 14034
    Root location: /home/artemiy/nn_project/data/seg_train/seg_train
    StandardTransform
Transform: Compose(
               Resize(size=(299, 299), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [75]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [58]:
train_dataset.class_to_idx

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [59]:
idx2class = {j: i for i, j in train_dataset.class_to_idx.items()}

In [60]:
idx2class

{0: 'buildings',
 1: 'forest',
 2: 'glacier',
 3: 'mountain',
 4: 'sea',
 5: 'street'}

In [61]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = torch.nn.CrossEntropyLoss()

In [62]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [63]:
len(idx2class)

6

In [66]:
model.fc = torch.nn.Linear(2048, 6)

In [67]:
model.to(DEVICE)
tu.get_model_summary(model, torch.randn(3, 3, 299, 299, device=DEVICE))

Layer                                              Kernel              Output          Params             FLOPs
0_Conv2d_1a_3x3.Conv2d_conv                       [3, 32, 3, 3]   [3, 32, 149, 149]         864      57,544,992
1_Conv2d_1a_3x3.BatchNorm2d_bn                             [32]   [3, 32, 149, 149]          64       8,525,184
2_Conv2d_2a_3x3.Conv2d_conv                      [32, 32, 3, 3]   [3, 32, 147, 147]       9,216     597,445,632
3_Conv2d_2a_3x3.BatchNorm2d_bn                             [32]   [3, 32, 147, 147]          64       8,297,856
4_Conv2d_2b_3x3.Conv2d_conv                      [32, 64, 3, 3]   [3, 64, 147, 147]      18,432   1,194,891,264
5_Conv2d_2b_3x3.BatchNorm2d_bn                             [64]   [3, 64, 147, 147]         128      16,595,712
6_maxpool1                                                    -     [3, 64, 73, 73]           0               0
7_Conv2d_3b_1x1.Conv2d_conv                      [64, 80, 1, 1]     [3, 80, 73, 73]       5,120      81,

In [68]:
for p in model.parameters():
    p.requires_grad = False

In [69]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [70]:
for p in model.fc.parameters():
    p.requires_grad = True

In [71]:
for p in model.parameters():
    print(p.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [72]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.002
    maximize: False
    weight_decay: 0
)

In [77]:
def fit_model(
    model: nn.Module,
    epochs: int,
    optimizer: torch.optim.Optimizer,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
    history=None,
) -> dict:
    """
    model: pytorch model - model to train
    epochs: int          - number of epochs
    optimizer: torch.optim.Optimizer - optimizer for training
    train_loader: DataLoader - DataLoader for training data
    valid_loader: DataLoader - DataLoader for validation data
    criterion: nn.Module - loss function
    device: torch.device - device to train on
    history: dict - dictionary to store training history
    """

    # будем сохранять значения точности и лосса в history
    history = history or {
        "train_accs": [],
        "train_losses": [],
        "valid_accs": [],
        "valid_losses": [],
    }

    # определяем текущую эпоху обучения
    start_epoch = len(history["train_accs"])
    for epoch in range(start_epoch + 1, start_epoch + epochs + 1):
        print(f'{"-"*13} Epoch {epoch} {"-"*13}')

        model.train()
        batch_accs = []
        batch_losses = []
        for samples, labels in train_loader:

            samples = samples.to(device)
            labels = labels.to(device)

            y_pred = model(samples)

            # Используем основной выход (logits) из InceptionOutputs
            if isinstance(y_pred, tuple):
                y_pred = y_pred.logits

            # Считаем лосс
            loss = criterion(y_pred, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_losses.append(loss.item())

            # сравниваем предсказания с таргетом и добавляем в
            # список значение точности
            batch_accs.append((y_pred.argmax(axis=1) == labels).cpu().numpy().mean())

        history["train_losses"].append(np.mean(batch_losses))
        history["train_accs"].append(np.mean(batch_accs))

        # Validation
        model.eval()

        batch_accs = []
        batch_losses = []
        for samples, labels in valid_loader:
            with torch.no_grad():
                y_pred = model(samples.to(device))

                # Используем основной выход (logits) из InceptionOutputs
                if isinstance(y_pred, tuple):
                    y_pred = y_pred.logits

            loss = criterion(y_pred, labels.to(device))
            batch_losses.append(loss.item())
            batch_accs.append(
                (y_pred.cpu().argmax(axis=1) == labels.cpu()).numpy().mean()
            )
        history["valid_accs"].append(np.mean(batch_accs))
        history["valid_losses"].append(np.mean(batch_losses))

        # печатаем результат

        print(
            f'train: accuracy {history["train_accs"][-1]:.4f}, loss {history["train_losses"][-1]:.4f}\n'
            f'valid: accuracy {history["valid_accs"][-1]:.4f}, loss {history["valid_losses"][-1]:.4f}'
        )
        print(f'{"-"*35}')
        print()

    return history

In [78]:
history1 = fit_model(
    model=model,
    epochs=10,
    optimizer=optimizer,
    train_loader=train_loader,
    valid_loader=valid_loader,
    criterion=criterion,
    device=DEVICE,
)

------------- Epoch 1 -------------
train: accuracy 0.1428, loss 1.8564
valid: accuracy 0.1301, loss 1.8381
-----------------------------------

------------- Epoch 2 -------------
train: accuracy 0.1495, loss 1.8499
valid: accuracy 0.1341, loss 1.8350
-----------------------------------

------------- Epoch 3 -------------
train: accuracy 0.1451, loss 1.8543
valid: accuracy 0.1323, loss 1.8366
-----------------------------------

------------- Epoch 4 -------------
train: accuracy 0.1440, loss 1.8526
valid: accuracy 0.1357, loss 1.8375
-----------------------------------

------------- Epoch 5 -------------
train: accuracy 0.1522, loss 1.8518
valid: accuracy 0.1355, loss 1.8372
-----------------------------------

------------- Epoch 6 -------------
train: accuracy 0.1468, loss 1.8542
valid: accuracy 0.1389, loss 1.8366
-----------------------------------

------------- Epoch 7 -------------
train: accuracy 0.1438, loss 1.8540
valid: accuracy 0.1316, loss 1.8361
----------------------